In [6]:
import os
import uuid
import shutil
#import pylatex
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from datetime import datetime
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Pool
#from pylatex.section import Chapter, Subsection
#from pylatex import Document, Section, Figure, NoEscape, Command

%run pipeline.ipynb

import warnings
warnings.filterwarnings("ignore")

In [7]:
lclist=pd.read_csv('lcnameswithperiods.csv')

In [8]:
plotprobs=np.array(['TIC-404335106-38-FFI-10min', 
          'TIC-300651846-13-FFI-30min',
          'TIC-442571495-91-FFI-200s',
          'TIC-442575691-91-FFI-200s',
          'TIC-244161191-42-FFI-10min',
          'EPIC-60019763-1-K2SFF-1800']) 

First try: just get the light curves for each of them. Returned light curve is just full of NaNs for all FFI ones and can't get a light curve for the last one. Deleting all the problematic FFI light curves and going to try redownloading them.

In [10]:
lc0=get_lc_string(plotprobs[0])

In [11]:
lc1=get_lc_string(plotprobs[1])
lc2=get_lc_string(plotprobs[2])
lc3=get_lc_string(plotprobs[3])
lc4=get_lc_string(plotprobs[4])

In [12]:
lc0

time,flux,flux_err
,,
Time,float64,float64
2333.8625264995426,nan,nan
2333.869470975828,nan,nan
2333.876415452113,nan,nan
2333.883359928398,nan,nan
2333.890304404682,nan,nan
2333.897248880967,nan,nan
2333.904193357718,nan,nan
2333.911137834002,nan,nan


They are all still nan even after redownloading and reprocessing them. Now will try to figure out what is going on within the actual source file.

In [14]:
#download_object(objectname='TIC 404335106', sector='38')

In [15]:
file='./data/tess-s0038-2-4_195.824160_-51.836880_50x50_astrocut.fits'

In [16]:
sourceobj=tess_cpm.Source(file, remove_bad=True)

Removing 12 bad data points (out of 3705) using the TESS provided QUALITY array


In [17]:
sourceobj.cutout_data.fluxes #this all the fluxes in the whole cutout

array([[[218.14436, 228.87123, 236.0626 , ..., 244.12936, 214.19653,
         212.85933],
        [252.32779, 312.28726, 259.6327 , ..., 236.37906, 215.06009,
         209.55182],
        [296.3421 , 413.1899 , 277.2135 , ..., 237.84648, 216.60255,
         213.53917],
        ...,
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan]],

       [[219.43092, 228.80064, 237.03696, ..., 244.58374, 214.07475,
         212.56519],
        [254.32596, 312.21738, 260.67636, ..., 234.52135, 217.12704,
         209.25769],
        [297.1889 , 413.12067, 277.6598 , ..., 237.14847, 217.58252,
         213.78317],
        ...,
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
   

In [18]:
sourceobj.cutout_data.fluxes.shape #seems like each data point mentioned above is 50x50
#since the large number is 3705 minus the 12 bad ones

(3693, 50, 50)

In [19]:
sourceobj.cutout_data.fluxes.size

9232500

In [20]:
np.count_nonzero(np.isnan(sourceobj.cutout_data.fluxes))

738600

In [21]:
np.count_nonzero(np.isnan(sourceobj.cutout_data.normalized_fluxes)) #basically 1000 of the data points are missing the normalized fluxes

2585100

In [22]:
sourceobj.cutout_data.flux_medians.shape

(50, 50)

In [23]:
np.count_nonzero(np.isnan(sourceobj.cutout_data.flux_medians.shape))

0

In [24]:
sourceobj.set_aperture(rowlims=[25, 25], collims=[25, 25]) #this narrows down the fluxes just to the aperture we are interested in
#and fill in the fluxes field for this object and not just the connected cutout_data object
#it specifically calls from the normalized_fluxes field of the sourceobj

In [25]:
sourceobj.models

[[<tess_cpm.model.PixelModel at 0x169265400>]]

In [26]:
np.count_nonzero(np.isnan(sourceobj.fluxes)) 

3693

Let's compare this number of nans to a normal sector.

In [28]:
#download_object(objectname='TIC 404335106', sector='11') #this sector plotted just fine for the same object

In [29]:
file2='./data/tess-s0011-2-4_195.824160_-51.836880_50x50_astrocut.fits'
sourceobj2=tess_cpm.Source(file2, remove_bad=True)

Removing 103 bad data points (out of 1248) using the TESS provided QUALITY array


In [30]:
np.count_nonzero(np.isnan(sourceobj2.cutout_data.normalized_fluxes)) #no nans!

0

In [31]:
#download_object(objectname='TIC 300651846', sector='1') #a fine sector for another problematic object

In [32]:
file3='./data/tess-s0001-4-3_114.366749_-66.757506_50x50_astrocut.fits'
sourceobj3=tess_cpm.Source(file3, remove_bad=True)

Removing 15 bad data points (out of 1282) using the TESS provided QUALITY array


In [33]:
np.count_nonzero(np.isnan(sourceobj3.cutout_data.normalized_fluxes)) #also no nans!

0

In [34]:
#download_object(objectname='TIC 300651846', sector='13') #the bad sector for this one

In [35]:
file4='./data/tess-s0013-4-2_114.366749_-66.757506_50x50_astrocut.fits'
sourceobj4=tess_cpm.Source(file4, remove_bad=True)

Removing 37 bad data points (out of 1320) using the TESS provided QUALITY array


In [36]:
np.count_nonzero(np.isnan(sourceobj4.cutout_data.normalized_fluxes)) #this one does have nans! since it's another bad one

898100

In [37]:
sourceobj4.set_aperture(rowlims=[25, 25], collims=[25, 25])

In [38]:
np.count_nonzero(np.isnan(sourceobj4.fluxes)) 

1283

In [39]:
lc5=get_lc_string(plotprobs[5]) #now the k2sff one. this is the only one that is in campaign one.

No data found for target "EPIC 60019763".


In [77]:
result=lk.search_lightcurve('EPIC 60019763')

In [79]:
result

#,mission,year,author,exptime,target_name,distance
,,,,s,,arcsec
0,K2 Campaign -1,2014,K2SFF,1800,ktwo060019763,0.0


In [82]:
result.download()

time,flux,flux_err,fraw,fcor,arclength,moving,cadenceno
,,,,,,,
Time,float64,float64,float64,float64,float64,int16,int32
1862.4001995200015,1.0069818111010655,nan,1.005514904570932,1.0069818111010655,1.8734760710515348,0,85813
1862.441066719999,0.9990943314297228,nan,0.998424921283779,0.9990943314297228,1.9563204041306064,0,85815
1862.4615003200015,0.9880083017821546,nan,0.9858023135320861,0.9880083017821546,1.7852466540552265,0,85816
1862.4819340199974,0.9821094673646457,nan,0.9753769113361568,0.9821094673646457,1.4873378343681445,0,85817
1862.5228012200023,0.9764736726986964,nan,0.9779950681527502,0.9764736726986964,2.7088676154743316,1,85819
1862.5432348199975,0.9809996228861267,nan,0.9832913154524658,0.9809996228861267,2.8055653510589718,0,85820
1862.5636683199991,0.9890864309187133,nan,0.990367253948768,0.9890864309187133,2.6761686818208044,0,85821
1862.5841020200023,0.9898819538784819,nan,0.9926392092797727,0.9898819538784819,2.4036382116297075,0,85822
